In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("ggplot")
import optuna

import seaborn as sns

In [ ]:
# データの読み込み
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train_=df_train
df_test_=df_test

df_train_["train_test"]="train"
df_test_["train_test"]="test"

df_test_.insert(1,"Survived",np.nan)

df_all=pd.concat([df_train_,df_test_])

df_all.reset_index(drop=True)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,train_test
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,train
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,train
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,train
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,train
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,test
1305,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,test
1306,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,test
1307,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,test


In [ ]:
df_all["honor"]=df_all["Name"].map(lambda x:x.split(",")[1].split(".")[0])
df_all["honor"]=df_all["honor"].replace((" Don"," Rev"," Jonkheer"," Capt"),1)
df_all["honor"]=df_all["honor"].replace((" Mr"),2)
df_all["honor"]=df_all["honor"].replace((" Master"," Dr"," Major"," Col"),3)
df_all["honor"]=df_all["honor"].replace((" Miss",),4)
df_all["honor"]=df_all["honor"].replace((" Mrs"," Mme"," Ms"," Lady"," Sir"," Mlle"," the Countess"," Dona"),5)
df_all["Namelen"]=df_all["Name"].map(lambda x: len(str(x)))
df_all["Nameini"]=df_all["Name"].map(lambda x: str(x)[0])

In [ ]:
df_all.groupby("honor")["Age"].median()
df_all.loc[(df_all["honor"]==1)&(df_all["Age"].isnull()),"Age"]=41.0
df_all.loc[(df_all["honor"]==2)&(df_all["Age"].isnull()),"Age"]=29.0
df_all.loc[(df_all["honor"]==3)&(df_all["Age"].isnull()),"Age"]=7.0
df_all.loc[(df_all["honor"]==4)&(df_all["Age"].isnull()),"Age"]=22.0
df_all.loc[(df_all["honor"]==5)&(df_all["Age"].isnull()),"Age"]=35.0




df_all["Family"]=df_all["Parch"]+df_all["SibSp"]+1


df_all["Ticketini"]=df_all["Ticket"].map(lambda x: str(x)[0])
df_all["Ticketlen"]=df_all["Ticket"].map(lambda x: len(str(x)))


df_all["Fare"]=df_all["Fare"].fillna(df_all["Fare"].median())

df_all["Embarked"]=df_all["Embarked"].fillna("S")

In [ ]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 417
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1309 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1309 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1309 non-null   object 
 12  train_test   1309 non-null   object 
 13  honor        1309 non-null   int64  
 14  Namelen      1309 non-null   int64  
 15  Nameini      1309 non-null   object 
 16  Family       1309 non-null   int64  
 17  Ticketini    1309 non-null   object 
 18  Ticketlen    1309 non-null   int64  
dtypes: float64(3

In [ ]:
from sklearn.preprocessing import LabelEncoder
lbl=LabelEncoder()
df_all["Sex"]=lbl.fit_transform(df_all["Sex"])
df_all["Embarked"]=lbl.fit_transform(df_all["Embarked"])
df_all["Ticketini"]=lbl.fit_transform(df_all["Ticketini"])
df_all["Nameini"]=lbl.fit_transform(df_all["Nameini"])

In [ ]:
df_all=df_all.drop(["Name","Ticket","Cabin"],axis=1)

In [ ]:
train=df_all[df_all["train_test"]=="train"]
testsub=df_all[df_all["train_test"]=="test"]

x=train.drop(["PassengerId","Survived","train_test"],axis=1)
t=train.iloc[:,1:2]
test=testsub.drop(["PassengerId","Survived","train_test"],axis=1)

In [ ]:
print(test.columns)

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'honor',
       'Namelen', 'Nameini', 'Family', 'Ticketini', 'Ticketlen'],
      dtype='object')


In [ ]:
import optuna
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from lightgbm import early_stopping
import optuna

# 訓練データの分割
x_train, x_test, y_train, y_test = train_test_split(x, t, test_size=0.2, random_state=0)

# 1. 目的関数の定義
def objective(trial):
    # ハイパーパラメータの探索範囲を定義
    hidden_layer_sizes_1 = trial.suggest_int('hidden_layer_sizes_1', 80, 200)
    hidden_layer_sizes_2 = trial.suggest_int('hidden_layer_sizes_2', 80, 200)
    hidden_layer_sizes_3 = trial.suggest_int('hidden_layer_sizes_3', 10, 100)
    learning_rate_init = trial.suggest_loguniform('learning_rate_init', 0.001, 0.1)

    # MLPClassifierモデルを定義
    clf1 = MLPClassifier(
        max_iter=10000,
        early_stopping=True,
        hidden_layer_sizes=[hidden_layer_sizes_1, hidden_layer_sizes_2, hidden_layer_sizes_3],
        learning_rate_init=learning_rate_init,
        solver='adam',
        activation='relu',
        random_state=0
    )

    # クロスバリデーションでのスコアを計算
    score = cross_val_score(clf1, x_train, y_train, cv=3)

    # 3つのスコアの平均を返す
    return np.mean(score)

# 2. Optunaオブジェクトを作成
study = optuna.create_study(direction='maximize')

# 3. Optunaでパラメータ探索
study.optimize(objective, n_trials=100)

# 4. ベストパラメータ表示
print("Best parameters found by Optuna:", study.best_params)

# 5. ベストパラメータでモデルを訓練
best_params = study.best_params
clf_best = MLPClassifier(
    max_iter=10000,
    early_stopping=True,
    hidden_layer_sizes=[best_params['hidden_layer_sizes_1'], best_params['hidden_layer_sizes_2'], best_params['hidden_layer_sizes_3']],
    learning_rate_init=best_params['learning_rate_init'],
    solver='adam',
    activation='relu',
    random_state=0
)

# ベストモデルで訓練
clf_best.fit(x_train, y_train)

# 訓練データの予測と精度
train_accuracy = clf_best.score(x_train, y_train)
print(f'Training Accuracy: {train_accuracy:.4f}')

# テストデータの予測と精度
test_accuracy = clf_best.score(x_test, y_test)
print(f'Test Accuracy: {test_accuracy:.4f}')


[I 2024-08-24 11:17:03,115] A new study created in memory with name: no-name-89ab89d5-f8da-4a00-9488-e456d7f1b870
<ipython-input-31-f99a7745d477>:21: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate_init = trial.suggest_loguniform('learning_rate_init', 0.001, 0.1)
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1102: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1102: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=T

Best parameters found by Optuna: {'hidden_layer_sizes_1': 130, 'hidden_layer_sizes_2': 183, 'hidden_layer_sizes_3': 95, 'learning_rate_init': 0.040672884983710224}
Training Accuracy: 0.8048
Test Accuracy: 0.8045


In [ ]:
# testデータの予測
test_predictions = clf_best.predict(test)
test_predictions = np.round(test_predictions, 0).astype(int)
# 提出用のデータフレームを作成
submission = pd.DataFrame({
    "PassengerId": testsub["PassengerId"],  # テストデータからPassengerIdを取得
    "Survived": test_predictions         # 予測結果をSurvived列に格納
})

# CSVファイルとして保存
submission.to_csv("submission.csv", index=False)

# 完了メッセージ
print("Submission file 'submission.csv' has been created and is ready for upload to Kaggle.")


Submission file 'submission.csv' has been created and is ready for upload to Kaggle.


In [ ]:
test_predictions

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,